In [ ]:
!pip install -q keras
!pip install keras-tuner --upgrade
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 5.6 MB/s 


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TrainingData_6000.csv')
df.head()

,DepthCut,SpindleSpd,Stable/Unstable
0,0.021,1800,1
1,0.022,1800,1
2,0.023,1800,1
3,0.024,1800,1
4,0.025,1800,1


In [ ]:
X=df.iloc[:,0:2]
y=df.iloc[:,2]

In this I am using kerasTuner to figure out the best set of hyperparameter for the model i.e Total number of the layers , number of neuron in each layer, optimal learning rate.

In [ ]:
def build_model(hp):
  model=keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):      #Choosing number of layers in range 2 to 20
    model.add(layers.Dense(hp.Int('units'+str(i),min_value=32,max_value=512,step=32),kernel_initializer='he_uniform',activation='relu')) #Number of neuron in each layer is between 32-512
  model.add(layers.Dense(1,kernel_initializer='glorot_uniform', activation='sigmoid'))
  model.compile(
      keras.optimizers.Adam(
   hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),loss='binary_crossentropy',metrics=['accuracy'])
  return model
  

In [ ]:
tuner = kt.RandomSearch(                            #RandomSearch func will search through the build_model 15 times with different hyperparameter
    build_model,                                    #each time with an objective to find accuracy and give us the best model (with best choice of num_layers, num_neuron)
    objective='val_accuracy',
    max_trials=15,
    executions_per_trial=1,
    directory='project',project_name='Ann'
    )

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
tuner.search(X_train, y_train,
             epochs=100,
             validation_data=(X_test, y_test))

Trial 15 Complete [00h 04m 23s]
val_accuracy: 0.6388534903526306

Best val_accuracy So Far: 0.9770700931549072
Total elapsed time: 00h 51m 42s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()


Results summary
Results in project/Ann
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 4
units0: 192
units1: 192
learning_rate: 0.01
units2: 192
units3: 224
units4: 480
units5: 416
units6: 256
units7: 192
units8: 64
units9: 448
units10: 480
units11: 480
units12: 256
units13: 384
units14: 352
units15: 320
units16: 384
units17: 64
Score: 0.9770700931549072
Trial summary
Hyperparameters:
num_layers: 9
units0: 384
units1: 480
learning_rate: 0.001
units2: 32
units3: 32
units4: 32
units5: 32
units6: 32
units7: 32
units8: 32
Score: 0.9719745516777039
Trial summary
Hyperparameters:
num_layers: 10
units0: 352
units1: 480
learning_rate: 0.001
units2: 224
units3: 224
units4: 480
units5: 512
units6: 256
units7: 480
units8: 288
units9: 128
units10: 288
units11: 256
units12: 384
units13: 64
units14: 384
Score: 0.9694267511367798
Trial summary
Hyperparameters:
num_layers: 18
units0: 448
units1: 512
learning_rate: 0.0001
units2: 448
units3: 192
units4: 64
units5: 320
units6: 160
unit

In [ ]:
models = tuner.get_best_models(num_models=2)    #with this we get the best model 
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 1,2)) #Input is depth of cut and spindle speed. Hence input_shape =(None,1,2)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 192)            576       
                                                                 
 dense_1 (Dense)             (None, 1, 192)            37056     
                                                                 
 dense_2 (Dense)             (None, 1, 192)            37056     
                                                                 
 dense_3 (Dense)             (None, 1, 224)            43232     
                                                                 
 dense_4 (Dense)             (None, 1, 1)              225       
                                                                 
Total params: 118,145
Trainable params: 118,145
Non-trainable params: 0
_________________________________________________________________
